In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(".."))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
# from sklearn.datasets import make_blobs

# from svm.soft_margin_svm import SoftMarginSVM
# from preprocessing.ohe import OneHotEncoder
# from linear_models.classification.softmax_regression import SoftMaxRegression
# from neighbors.knn_classifier import KNeighborsClassifier
# from linear_models.classification.logistic_regression import LogisticRegression
# from linear_models.regression.linear_regression import LinearRegression
# from linear_models.classification.least_squares_classifier import LeastSquareClassifier
# from classification.perceptron import Perceptron

In [2]:
def visualize_model(x_train, labels, model, degree = 1):
    f = plt.figure(figsize = (8,8))
    x1_min = np.min(x_train[:,0])
    x1_max = np.max(x_train[:,0])
    x2_min = np.min(x_train[:,1])
    x2_max = np.max(x_train[:,1])
    
    x1_test, x2_test = np.meshgrid(np.linspace(x1_min, x1_max, 100), 
                                   np.linspace(x2_min, x2_max, 100))
    x_test_poly = np.array([x1_test, x2_test]).reshape(2, -1).T
    y_test = model.predict(x_test_poly)

    flat_labels = labels.flatten()
    sns.scatterplot(data = x_train, x = x_train[:,0], y = x_train[:,1], hue = labels)
    plt.contourf(x1_test, x2_test, y_test.reshape(100,100), alpha = 0.5, levels = np.linspace(0,1,3))
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.xlim(x1_min, x1_max)
    plt.ylim(x2_min, x2_max)
    plt.gca().set_aspect('equal', adjustable = 'box')
    plt.show()

In [3]:
from trees.decision_tree_classifier import DecisionTreeClassifier

In [4]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y = True)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state = 42)

In [5]:
dt = DecisionTreeClassifier(max_depth = 10)
dt.fit(xtrain, ytrain)

In [6]:
yhat = dt.predict(xtest)

In [7]:
dt.print_tree()

if Attr 2 <= 1.9
 left: --> (Class: 0.0)
 right: if Attr 2 <= 4.7
  left: if Attr 3 <= 1.6
    left: --> (Class: 1.0)
    right: --> (Class: 2.0)
  right: if Attr 3 <= 1.7
    left: if Attr 2 <= 4.9
        left: --> (Class: 1.0)
        right: if Attr 3 <= 1.5
                left: --> (Class: 2.0)
                right: --> (Class: 1.0)
    right: if Attr 2 <= 4.8
        left: --> (Class: 2.0)
        right: --> (Class: 2.0)
